In [0]:
# #Setup Notebook Parameters (Defaulting to Payment Pending and Running all tests)

#Imports
import json
import sys
import os
from datetime import datetime, timedelta
%pip install openpyxl

dbutils.widgets.text("all_test_results", "")
all_test_results_string = dbutils.widgets.get("all_test_results_string")

dbutils.widgets.text("state_under_test", "")
state_under_test = dbutils.widgets.get("state_under_test")

dbutils.widgets.text("base_path", "")
base_path = dbutils.widgets.get("base_path")

dbutils.widgets.text("test_results_path", "")
test_results_path = dbutils.widgets.get("test_results_path")
#new
dbutils.widgets.text("run_user", "")
run_user = dbutils.widgets.get("run_user")

dbutils.widgets.text("run_tag", "")
run_tag = dbutils.widgets.get("run_tag")

dbutils.widgets.text("run_by_automation_name", "")
run_by_automation_name = dbutils.widgets.get("run_by_automation_name")

dbutils.widgets.text("run_start_datetime", "")
run_start_datetime_string = dbutils.widgets.get("run_start_datetime")
run_start_datetime = datetime.strptime(run_start_datetime_string, "%Y-%m-%d %H:%M:%S")


# Add base_path to sys.path to find models
if base_path not in sys.path:
    sys.path.append(base_path)
# from models.test_result import TestResult

#Add as short term solution to above that stopped working
from dataclasses import dataclass
@dataclass
class TestResult:
    test_field: str =""     
    status: str =""
    message: str = ""
    test_from_state: str=""
    test_name: str=""

all_test_results = [TestResult(**d) for d in json.loads(all_test_results_string)]


###############################
#Update Central Results Table
###############################
import uuid
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp
from pyspark.sql.types import *
from datetime import datetime, timedelta

#Define Schema
def run_and_result_table_schemas():

    runs_schema = StructType([
        StructField("run_id", StringType(), True),
        StructField("run_user", StringType(), True),
        StructField("run_start_datetime", TimestampType(), True),
        StructField("run_end_datetime", TimestampType(), True),    
        StructField("run_by_automation_name", StringType(), True),
        StructField("run_tag", StringType(), True),
        StructField("run_status", StringType(), True),
        StructField("state_under_test", StringType(), True), 
        StructField("total_passed", IntegerType(), True), 
        StructField("total_failed", IntegerType(), True), 
        StructField("total_tests", IntegerType(), True), 
    ])

    results_schema = StructType([
        StructField("result_id", StringType(), True),
        StructField("run_id", StringType(), True),  # FK to test_runs
        StructField("test_name", StringType(), True),
        StructField("test_field", StringType(), True),
        StructField("test_from_state", StringType(), True),
        StructField("status", StringType(), True),
        StructField("message", StringType(), True)
    ])
    return runs_schema, results_schema


###############################
#Create Run First (so can get ID to register results against)
###############################
def create_run(run_user, run_start_datetime, run_end_datetime, run_by_automation_name, run_tag, run_status,state_under_test,pass_count, fail_count, total_count):        
    try:
        run_id = str(uuid.uuid4())
        runs_schema, results_schema = run_and_result_table_schemas()
        df = spark.createDataFrame(
            [(run_id, run_user, run_start_datetime, run_end_datetime, run_by_automation_name, run_tag, run_status,state_under_test,pass_count, fail_count, total_count)],runs_schema)
        df.show()
        print("After Create dataframe)")
        df.write.option("mergeSchema", "true").mode("append").saveAsTable("test_automation_runs")    
        print("After Write dataframe)")
        return run_id
    except Exception as e:
        error_message = str(e)  
        print(f"Failed to Create New Run Record. error : {message}")  
        return None
          
###############################
#Create Each Test Result
###############################
def create_result(run_id):
    try:
        runs_schema, results_schema = run_and_result_table_schemas()
        rows = []        
        for result in all_test_results:                               
                rows.append(
                    (
                    str(uuid.uuid4()),
                    run_id,
                    str(getattr(result, "test_name", "") or ""),
                    str(getattr(result, "test_field", "") or ""),
                    str(getattr(result, "test_from_state", "") or ""),
                    str(getattr(result, "status", "") or ""),
                    str(getattr(result, "message", "") or "")
                    )
                )        
        df = spark.createDataFrame(rows, results_schema)
        df.write.option("mergeSchema", "true").mode("append").saveAsTable("test_automation_results")    
    except Exception as e:
        error_message = str(e)   
        print(f"failed to Update Record : {result.test_name} - error : {message}")




In [0]:
#################################
#Produce File Test Reports
#################################

# Count passed and failed
pass_count = sum(1 for r in all_test_results if r.status.upper() == "PASS")
fail_count = sum(1 for r in all_test_results if r.status.upper() == "FAIL")
run_status = "PASS" if fail_count == 0 and pass_count >= 1 else "FAIL"
print(f"OVERALL TEST RESULTS - Status :  {run_status} - FOR STATE : {state_under_test} -- Pass: {pass_count}, Fail: {fail_count}")

#Display / Output Test results
import pandas as pd
import openpyxl

#convert list of dicts string into df
df_results = pd.DataFrame(eval(all_test_results_string))
display(df_results)  # Databricks display

#Export Results
from datetime import datetime
now = datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")

#Create Results Folder
test_results_path = f"{test_results_path}/{timestamp}-{state_under_test}/"
os.makedirs(test_results_path, exist_ok=True)

#TODO - Choose output format

# Export to CSV
file_path = f"{test_results_path}/test_results_{state_under_test}_{timestamp}.csv"
df_results.to_csv(file_path, index=False)
print(f"Test results saved to {file_path}")

# Export to XLS
file_path = f"{test_results_path}/test_results_{state_under_test}_{timestamp}.xlsx"
df_results.to_excel(file_path, engine="openpyxl", index=False )
print(f"Test results saved to {file_path}")

#Push Results into Spark Table
# from table_utils import create_run, create_result
print("Starting Pushing Run/Results into Tables")
run_end_datetime = datetime.now()
run_id = create_run(run_user, run_start_datetime, run_end_datetime, run_by_automation_name, run_tag, run_status,state_under_test,pass_count, fail_count, pass_count + fail_count)
print(f"Finsihed creating Run -- Run_id = {str(run_id)}")
if run_id != None:
    create_result(run_id)
    print(f"Finsihed creating Results")
else:
    print("Failed to Create a Run, No results have been submitted to spark tables")
    

# Export to HTML
file_path = f"{test_results_path}/test_results_{state_under_test}_{timestamp}.html"
def color_status(val):
    color = 'green' if val == 'PASS' else 'red'
    return f'color: {color}; font-weight: bold'

# Convert DataFrame to styled HTML
styled_html = df_results.style.applymap(color_status, subset=['status']) \
                    .set_table_styles([
                        {'selector': 'table', 'props': [('border-collapse', 'collapse'), 
                                                        ('width', '80%')]},
                        {'selector': 'th, td', 'props': [('border', '1px solid black'),
                                                        ('padding', '8px'),
                                                        ('text-align', 'left')]}
                    ]) \
                    .render()

# Wrap in basic HTML tags
html_content = f"""
<html>
<head>
    <title>Test Results - {state_under_test} - {timestamp}</title>
</head>
<body>
    <h2>Test Results - {state_under_test}- {timestamp}</h2>
    <p><strong>Total PASS:</strong> {str(pass_count)} &nbsp;&nbsp; <strong>Total FAIL:</strong> {str(fail_count)}</p>
    {styled_html}
</body>
</html>
"""

with open(file_path, "w") as f:
    f.write(html_content)
print(f"HTML report saved to {file_path}")